# TSP Algorithm Analysis

4-plot comparison of TSP algorithms:
- Cost vs Time (1 second max)
- Cost vs Iterations (1000 iterations max) 
- Final Cost Comparison
- Steps per Second

In [ ]:
from tsp.io import parse_tsplib_tsp
from algorithm.nearest_neighbor import NearestNeighbor
from algorithm.random_solver import RandomSolver
from pathlib import Path
import time
import matplotlib.pyplot as plt


print("Setup complete!")

In [ ]:
# Load instance
instance = parse_tsplib_tsp(Path("dataset/berlin52.tsp"))
print(f"Loaded {instance.name} with {len(instance.cities)} cities")

# Check for optimal tour file
opt_tour_path = Path(f"dataset/{instance.name}.opt.tour")
optimal_cost = None
if opt_tour_path.exists():
    print(f"Found optimal tour file: {opt_tour_path}")
    # Parse optimal tour (simple format: just city indices)
    with open(opt_tour_path, "r") as f:
        lines = f.readlines()
    
    # Find TOUR_SECTION and read tour
    in_tour = False
    tour = []
    for line in lines:
        line = line.strip()
        if line.upper().startswith("TOUR_SECTION"):
            in_tour = True
            continue
        if line.upper().startswith("EOF") or line == "-1":
            break
        if in_tour and line:
            try:
                # Convert 1-based TSPLIB ids to 0-based indices
                city_id = int(line) - 1
                tour.append(city_id)
            except ValueError:
                continue
    
    if tour:
        optimal_cost = instance.route_cost(tour)
        print(f"Optimal cost: {optimal_cost:.2f}")
    else:
        print("Could not parse optimal tour")
else:
    print("No optimal tour file found")

In [ ]:
def run_algorithm_with_timing(instance, solver, initial_route, max_seconds=1.0):
    """Run algorithm for fixed time and collect cost at each step."""
    solver.initialize(initial_route)
    
    iterations = []
    costs = []
    times = []
    
    start_time = time.perf_counter()
    
    while time.perf_counter() - start_time < max_seconds:
        report = solver.step()
        current_time = time.perf_counter() - start_time
        
        iterations.append(report.iteration)
        costs.append(report.cost)
        times.append(current_time)
    
    return iterations, costs, times


def run_algorithm_with_iterations(instance, solver, initial_route, max_iterations=1000):
    """Run algorithm for fixed number of iterations and collect cost at each step."""
    solver.initialize(initial_route)
    
    iterations = []
    costs = []
    times = []
    
    start_time = time.perf_counter()
    
    for i in range(max_iterations):
        report = solver.step()
        current_time = time.perf_counter() - start_time
        
        iterations.append(report.iteration)
        costs.append(report.cost)
        times.append(current_time)
    
    return iterations, costs, times


In [ ]:
# Initial route (identity permutation)
seed_route = list(range(len(instance.cities)))

# Run algorithms for time-based benchmark
algorithms = {
    "Nearest Neighbor": NearestNeighbor(instance),
    "Random": RandomSolver(instance, seed=42)
}

time_results = {}
for name, solver in algorithms.items():
    print(f"Running {name} for 1 second...")
    iterations, costs, times = run_algorithm_with_timing(instance, solver, seed_route, 1.0)
    time_results[name] = {
        'iterations': iterations,
        'costs': costs,
        'times': times,
        'final_cost': costs[-1] if costs else float('inf'),
        'total_iterations': len(iterations)
    }
    print(f"  Final cost: {time_results[name]['final_cost']:.2f}")
    print(f"  Iterations: {time_results[name]['total_iterations']}")


In [ ]:
# Run algorithms for iteration-based benchmark
iteration_results = {}
for name, solver in algorithms.items():
    print(f"Running {name} for 1000 iterations...")
    iterations, costs, times = run_algorithm_with_iterations(instance, solver, seed_route, 1000)
    iteration_results[name] = {
        'iterations': iterations,
        'costs': costs,
        'times': times,
        'final_cost': costs[-1] if costs else float('inf'),
        'total_time': times[-1] if times else 0
    }
    print(f"  Final cost: {iteration_results[name]['final_cost']:.2f}")
    print(f"  Total time: {iteration_results[name]['total_time']:.3f} seconds")


In [ ]:
# Create comprehensive plot
plt.figure(figsize=(12, 8))

# Plot 1: Cost vs Time (1 second max)
plt.subplot(2, 2, 1)
for name, data in time_results.items():
    plt.plot(data['times'], data['costs'], label=name, linewidth=2)
if optimal_cost is not None:
    plt.axhline(y=optimal_cost, color='green', linestyle='--', alpha=0.7, label='Optimal')
plt.xlabel('Time (seconds)')
plt.ylabel('Cost')
plt.title('Cost vs Time (1 second max)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Cost vs Iterations (1000 iterations max)
plt.subplot(2, 2, 2)
for name, data in iteration_results.items():
    plt.plot(data['iterations'], data['costs'], label=name, linewidth=2)
if optimal_cost is not None:
    plt.axhline(y=optimal_cost, color='green', linestyle='--', alpha=0.7, label='Optimal')
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.title('Cost vs Iterations (1000 iterations max)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Final Cost Comparison
plt.subplot(2, 2, 3)
names = list(time_results.keys())
final_costs = [time_results[name]['final_cost'] for name in names]

# Add optimal cost if available
if optimal_cost is not None:
    names.append('Optimal')
    final_costs.append(optimal_cost)
    colors = ['blue', 'red', 'green']  # NN, Random, Optimal
else:
    colors = ['blue', 'red']  # NN, Random

bars = plt.bar(names, final_costs, color=colors)
plt.ylabel('Final Cost')
plt.title('Final Cost Comparison')
plt.xticks(rotation=45)

# Add value labels on bars
for bar, cost in zip(bars, final_costs):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
            f'{cost:.0f}', ha='center', va='bottom')

# Add padding to y-axis to prevent label cutoff
y_max = max(final_costs)
plt.ylim(0, y_max * 1.15)  # 15% padding above the highest bar

# Plot 4: Steps per Second (bar chart)
plt.subplot(2, 2, 4)
algorithm_names = list(time_results.keys())
steps_per_second = [time_results[name]['total_iterations'] / 1.0 for name in algorithm_names]  # 1 second duration

bars = plt.bar(algorithm_names, steps_per_second, color=['blue', 'red'])
plt.ylabel('Steps per Second')
plt.title('Steps per Second')
plt.xticks(rotation=45)

# Add value labels on bars
for bar, rate in zip(bars, steps_per_second):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000, 
            f'{rate:.0f}', ha='center', va='bottom')

plt.tight_layout()

results_dir = Path("bench_results")
results_dir.mkdir(exist_ok=True)
timestamp = time.strftime("%Y%m%d_%H%M%S")
plot_file = results_dir / f"algorithm_comparison_{timestamp}.png"
plt.savefig(plot_file, dpi=300, bbox_inches='tight')
print(f"Plot saved as '{plot_file}'")

plt.show()
